In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().master("local[2]").getOrCreate()

In [3]:
graph_schema = StructType([
    StructField("to", IntegerType(), False),
    StructField("from", IntegerType(), False)
])

In [4]:
dist_schema = StructType([
    StructField("vertex", IntegerType(), False),
    StructField("distance", IntegerType(), False)
])

In [5]:
def shortest_path(v_from, v_to, dataset_path=None):
    edges = spark.read.csv(dataset_path, sep="\t", schema=graph_schema).cache()
    path_length = None
    
    # Here you should implement the BFS algorithm. It should return the length
    # of the minimal path (single integer) between v_from and v_to
    distances = edges.where(edges["from"]==12)\
    .withColumn("distance",lit(0))\
    .withColumnRenamed("to","to_d")\
    .withColumnRenamed("from","from_d")
    while True:
        new_distances = distances.join(edges,distances["to_d"] == edges["from"])\
        .withColumn("distance_t",col("distance")+1)\
        .drop("distance")\
        .withColumnRenamed("distance_t","distance")\
        .select("distance","to","from")\
        .withColumnRenamed("from","from_d")\
        .withColumnRenamed("to","to_d").persist()

        result = new_distances.where(new_distances["from_d"]==v_to).select("distance").take(1)
        if len(result)>0:
            path_length=result[0].distance
            break
        else:
            distances=new_distances
    
    return path_length

In [6]:
print shortest_path(12, 34, "/data/twitter/twitter_sample2.txt")

8
